In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
def get_page_content_to_sp(request_url):
    #print(request_url)
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}
    html = requests.get(request_url, headers=headers, timeout=10)
    content = html.text
    #print(content)
    soup = BeautifulSoup(content, 'html.parser', from_encoding='utf-8')
    return soup

def get_page_content(request_url):
    #print(request_url)
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}
    html = requests.get(request_url, headers=headers, timeout=10)
    content = html.text
    return content

def analysis(soup):
    temp = soup.find('div', class_='tslb_b')
    #df = pd.DataFrame(columns=['投诉编号','投诉品牌','投诉车系','投诉车型','问题简述','典型问题','投诉时间','投诉状态'])
    tr_list = temp.find_all('tr')
    #print(tr_list[0])
    columns = [i.string for i in tr_list[0].find_all('th')]
    #print(columns)
    df = pd.DataFrame(columns=columns)
    for tr in tr_list[1:]:
        data = [i.string for i in tr.find_all(['td', 'em']) if (i.find('img')) == None ]
        df.loc[len(df)] = data
    return df

def product_covert(x):
    problem = ''
    for i in x.split(','):
        problem += str(type_dict[i]) + ' ' if i in type_dict  else ''
    return problem
        
base_url = 'http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-0-0-0-0-0-'
type_url = 'http://www.12365auto.com/js/cTypeInfo.js?version=' + time.strftime("%Y%m%d", time.localtime()) 
result = pd.DataFrame()
type_info = get_page_content(type_url).split('=')[1]
type_js = eval(type_info)
type_dict = {}
for i in type_js:
    for j in i['items']:
        type_dict[i['value'] + str(j['id'])] = i['name']+str(':') + j['title']
page_num = 10
for i in range(page_num):
    soup = get_page_content_to_sp(base_url+str(i+1)+'.shtml')
    df = analysis(soup)
    result = result.append(df)
result['典型问题'] = result['典型问题'].apply(lambda x : product_covert(x)) 
result.head()
result.to_csv('车质网数据.csv',index=False)

In [24]:
import requests
from lxml import etree
import pandas as pd
res = requests.get('http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-0-0-0-0-0-1.shtml')
res_elements = etree.HTML(res.text)
table = res_elements.xpath('//table[@class="ar_c ar_c1"]')
table = etree.tostring(table[0], encoding='utf-8').decode()
df = pd.read_html(table, encoding='utf-8', header=0)[0]
df['典型问题'] = df['典型问题'].apply(lambda x : product_covert(x))
result.to_csv('车质网数据1.csv',index=False)